In [2]:
import pandas as pd
import numpy as np

In [ ]:
players = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv")

defensive_positions = ['CB', 'DE', 'OLB', 'FS', 'DT', 'SS', 'ILB', 'NT', 'MLB', 'DB', 'LB']
defensive_players_df = players[players['officialPosition'].isin(defensive_positions)]
defensive_players_list = defensive_players_df['nflId']

# We only want one row for each play which is where the event is ball snap
# The following code reads in each dataset for weeks 1-8 and only keeps rows where the event is ball snap AND 
#     where the nflid is a defensive player
loc = pd.DataFrame()
for i in range(1,8):
    url = 'https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/week'+str(i)+'.csv'
    week_data = pd.read_csv(url)
    week_data = week_data[week_data['event'] == 'ball_snap']
    week_data = week_data[(week_data['nflId'].isin(defensive_players_list)) | (week_data['team'] == 'football')]
    loc = pd.concat([loc, week_data])

In [ ]:
play_ids = loc['playId'].unique()

### Calculating the priors for each team

In [4]:
plays = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv')

In [5]:
len(plays)

8558

In [6]:
plays = plays[plays['playId'].isin(play_ids)]

In [15]:
plays.loc[(plays['gameId'] == 2021091212) & (plays['playId'] == 912)]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
958,2021091212,912,(11:19) (Shotgun) D.Jones pass incomplete shor...,2,3,7,NYG,DEN,NYG,45,...,NaN,65.0,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,"2 DL, 3 LB, 6 DB",NaN,0,Cover-1,Man


In [59]:
plays.groupby(['defensiveTeam','pff_passCoverage']).size().unstack(fill_value=0)

pff_passCoverage,2-Man,Bracket,Cover-0,Cover-1,Cover-2,Cover-3,Cover-6,Goal Line,Miscellaneous,Prevent,Quarters,Red Zone
defensiveTeam,,,,,,,,,,,,
ARI,16,0,18,37,35,89,22,4,1,1,26,6
ATL,5,2,6,46,107,43,4,0,1,0,1,7
BAL,7,2,21,78,16,75,34,1,0,0,20,4
BUF,11,1,4,54,26,54,18,0,0,0,63,12
CAR,2,1,9,70,26,82,14,2,0,0,40,13
CHI,1,0,9,63,14,67,57,0,0,0,12,16
CIN,4,1,14,73,58,101,30,0,0,0,28,12
CLE,9,1,1,49,1,89,16,3,0,0,77,12
DAL,7,0,4,96,40,60,2,0,0,3,31,8


In [60]:
plays = plays[plays.pff_passCoverage != 'Bracket']
plays = plays[plays.pff_passCoverage != 'Goal Line']
plays = plays[plays.pff_passCoverage != 'Miscellaneous']
plays = plays[plays.pff_passCoverage != 'Prevent']
plays = plays[plays.pff_passCoverage != 'Red Zone']

In [61]:
len(plays['playId'].unique())

3532

In [62]:
team_d = plays.groupby(['defensiveTeam','pff_passCoverage']).size().unstack(fill_value=0)

In [63]:
priors = team_d.div(team_d.sum(axis=1), axis=0)

In [1]:
priors

NameError: name 'priors' is not defined